## [Commands](https://ardupilot.org/dev/docs/copter-commands-in-guided-mode.html)

In [8]:
from pymavlink import mavutil
import subprocess
import time
import os

Kill related process

In [9]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Launch Copter (ardupilot)

In [10]:
sim_vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")
vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduCopter" 
# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

Conect to the device

In [11]:
conn = mavutil.mavlink_connection('udp:127.0.0.1:14550')

conn.wait_heartbeat()
print("Heartbeat received. Connected to the vehicle.")

Heartbeat received. Connected to the vehicle.


In [12]:
from mission_flow import UAVStep


ModuleNotFoundError: No module named 'mission_flow'

In [ ]:
def check_noarmed(blocking=False):
    """Check if UAV is already armed"""
    msg = conn.recv_match(type="HEARTBEAT", blocking=blocking)
    if not msg:
        return False  # No message received
    is_armed = msg.base_mode & mavutil.mavlink.MAV_MODE_FLAG_SAFETY_ARMED
    return not is_armed

check_noarmed_step = UAVStep(
    name="Check no armed",   
    check_fn=check_noarmed   # Your function goes here
)
check_noarmed_step.run(blocking=True)
check_noarmed_step.run(blocking=True)
print(check_noarmed_step)

▶️  Starting step: Check no armed
✅ Step done: Check no armed
<UAVStep name='Check no armed', state=DONE, next='None'>


In [7]:

# # ----- Action Class -----
# class UAVAction:
#     def __init__(self, name: str):
#         self.name = name
#         self.current_step = None
#         self.next = None

#     def add_step(self, name, check_fn=None, exec_fn=None):
#         step_number = len(self.steps)
#         step = UAVStep(name, step_number, check_fn, exec_fn)

#         if self.steps:
#             self.steps[-1].next = step
#             step.prev = self.steps[-1]

#         self.steps.append(step)

#         if self.current_step is None:
#             self.current_step = step

#     def link_next(self, next_action):
#         self.next_action = next_action
#         next_action.prev_action = self

#     def reset(self):
#         self.current_step = self.steps[0] if self.steps else None
#         self.state = ACTION_NOT_STARTED

#     def show_steps(self):
#         print(f"\n🧭 Action: {self.name} — {self.state}")
#         for step in self.steps:
#             current = "➡️ " if step == self.current_step else "   "
#             print(f"{current}{step}")

#     def run(self, blocking=True):
#         print(f"\n🚀 Starting action: {self.name}")
#         self.state = ACTION_IN_PROGRESS

#         while self.current_step:
#             step = self.current_step
#             print(f"➡️ Executing: {step}")

#             if not step.check():
#                 print(f"❌ Check failed: {step.name}")
#                 self.state = ACTION_FAILED
#                 return False

#             step.execute(blocking=blocking)
#             print(f"✅ Executed: {step.name}\n")
#             self.current_step = step.next

#         self.state = ACTION_DONE
#         print(f"✅ Action '{self.name}' completed!")
#         return True

#     def run_full(self, blocking=True):
#         current = self
#         while current:
#             current.reset()
#             if not current.run(blocking=blocking):
#                 print("❌ Mission flow interrupted.")
#                 return False
#             current = current.next_action
#         print("🎉 All actions completed successfully!")
#         return True

#     def show_status(self):
#         print(f"📦 Action '{self.name}': {self.state}")

In [8]:
REQUIRED_SENSORS = (
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_GYRO |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_ACCEL |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_MAG |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_ABSOLUTE_PRESSURE |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_GPS
)

EKF_REQUIRED_FLAGS = (
    mavutil.mavlink.EKF_ATTITUDE |
    mavutil.mavlink.EKF_VELOCITY_HORIZ |
    mavutil.mavlink.EKF_POS_HORIZ_ABS |
    mavutil.mavlink.EKF_POS_VERT_ABS
)


class UAV():
    def __init__(self,conn):
        self.conn=conn
        
    def check_heartbeat(blocking=False):
        """Check if UAV is already armed"""
        msg = conn.recv_match(type="HEARTBEAT", blocking=blocking)
        if msg:
            if msg.base_mode & mavutil.mavlink.MAV_MODE_FLAG_SAFETY_ARMED:
                print("❌ UAV is already armed!")
                return False
            print("✅ UAV is disarmed and ready to arm")
            return True
        return False

    def check_ekf_status(blocking=False):
        """Check EKF status for safe arming"""
        msg = conn.recv_match(type="EKF_STATUS_REPORT", blocking=blocking)
        if msg:
            ekf_flags = msg.flags
            # Required EKF flags
            if (ekf_flags & EKF_REQUIRED_FLAGS) == EKF_REQUIRED_FLAGS:
                print("✅ EKF is stable and ready for arming")
                return True
            else:
                print("❌ EKF not ready!")
        return False

    def check_gps_status(self,blocking):
        """Check GPS fix type"""
        msg = self.conn.recv_match(type="GPS_RAW_INT", blocking=blocking)
        if msg:
            gps_fix = msg.fix_type
            if gps_fix >= 3:  # 3D Fix or better
                print(f"✅ GPS Fix OK (Type {gps_fix})")
                return True
            else:
                print(f"❌ GPS Fix is weak (Type {gps_fix})")
        return False

    def check_sys_status(self,blocking):
        """Check system status (failsafe, battery, sensors)"""
        msg = self.conn.recv_match(type="SYS_STATUS", blocking=blocking)
        if msg:
            # Battery check
            if msg.battery_remaining > 20:  # More than 20% battery
                print(f"✅ Battery OK ({msg.battery_remaining}% remaining)")
                return True
            else:
                print(f"❌ Battery too low ({msg.battery_remaining}% remaining)")
            
            # Sensor check
            if msg and (msg.onboard_control_sensors_health & REQUIRED_SENSORS) == REQUIRED_SENSORS:
                print("✅ All required sensors are healthy! Ready for GPS-based autonomous flight")
                return True
            else:
                print("❌ Missing or unhealthy sensors! Not safe for autonomous flight")
        return False




## Prearmed Comands

In [9]:
# Run all pre-arm checks
uav=UAV(conn)

while True:
    if uav.check_heartbeat():
        break
while True:
    if uav.check_sys_status():
        break
while True:
    if uav.check_ekf_status():
        break
while True:
    if uav.check_gps_status():
        break

print("\n✅✅✅ UAV is READY TO ARM!\n")


✅ UAV is disarmed and ready to arm


TypeError: check_sys_status() missing 1 required positional argument: 'blocking'

## Movement Commands

These commands can be used to control the vehicle’s position, velocity or attitude while in [Guided Mode](https://ardupilot.org/copter/docs/ac2_guidedmode.html#ac2-guidedmode)

In [ ]:
mavutil.mavlink.EKF_POS_HORIZ_ABS

16